## Homework

In [63]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
'https://docs.google.com/uc?export=download: Неподдерживаемая схема.
'id' is not recognized as an internal or external command,
operable program or batch file.


В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [100]:
import pandas as pd

In [101]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3959 entries, 4 to 19997
Columns: 292 entries, id to price
dtypes: datetime64[ns](1), float64(119), int64(157), object(15)
memory usage: 8.8+ MB


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [102]:
import numpy as np

from sklearn.model_selection import train_test_split

y = np.array(df['price'])


In [103]:
drop_columns = [
    'id',  # May leak information
    'timestamp',  # May leak information
]
cat_columns = [
    'product_type',  #
    'material',  # Material of the wall
    'state',  # Satisfaction level
    'sub_area',  # District name
    'culture_objects_top_25',  #
    'thermal_power_plant_raion',  #
    'incineration_raion',  #
    'oil_chemistry_raion',  #
    'radiation_raion',  #
    'railroad_terminal_raion',  #
    'big_market_raion',  #
    'nuclear_reactor_raion',  #
    'detention_facility_raion',  #
    'ID_metro',  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',  #
    'ID_big_road1',  #
    'big_road1_1line',  #
    'ID_big_road2',  #
    'railroad_1line',  #
    'ID_railroad_terminal',  #
    'ID_bus_terminal',  #
    'ecology',  #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [107]:
X = df.drop(columns=['price', 'id', 'timestamp'])
X = pd.get_dummies(X, columns=cat_columns)
X = np.array(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [112]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

decision_tree_reg = DecisionTreeRegressor(max_depth=5, random_state=42)
decision_tree_reg.fit(X_train, y_train)


DecisionTreeRegressor(max_depth=5, random_state=42)

Проверьте качество на отложенной выборке.

In [113]:
print(mean_squared_error(y_test, decision_tree_reg.predict(X_test)))
print(decision_tree_reg.score(X_test, y_test))


13316211063079.395
0.5414886717047369


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [114]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

C:\Users\poisk\AppData\Local\Temp\ipykernel_10852\3560215100.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [115]:
df['month'] = pd.DatetimeIndex(df['timestamp']).month
df['weekday'] = df['timestamp'].dt.dayofweek
df['floor_divided_by_max_floor'] = (df['floor'].astype(float)) / (df['max_floor'].astype(float))
df['kitchenArea_divided_by_fullArea'] = (df['kitch_sq'].astype(float)) / (df['full_sq'].astype(float))
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.mean(), inplace=True)


C:\Users\poisk\AppData\Local\Temp\ipykernel_10852\2813260639.py:6: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df.fillna(df.mean(), inplace=True)
C:\Users\poisk\AppData\Local\Temp\ipykernel_10852\2813260639.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True)


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [116]:
y = np.array(df['price'])
X = df.drop(columns=['price', 'id', 'timestamp'])
X = pd.get_dummies(X, columns=cat_columns)
X = np.array(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [120]:
import catboost
from sklearn.ensemble import RandomForestRegressor

tree_reg = DecisionTreeRegressor(max_depth=5, random_state=42).fit(X_train, y_train)
forest_regressor = RandomForestRegressor(random_state=42).fit(X_train, y_train)
cat = catboost.CatBoostRegressor().fit(X_train, y_train)


Learning rate set to 0.049123
0:	learn: 6005286.6768326	total: 15ms	remaining: 15s
1:	learn: 5876494.3751646	total: 30.3ms	remaining: 15.1s
2:	learn: 5754418.8696846	total: 44ms	remaining: 14.6s
3:	learn: 5625106.0287390	total: 58.7ms	remaining: 14.6s
4:	learn: 5496430.8127954	total: 73.1ms	remaining: 14.5s
5:	learn: 5378099.1128402	total: 88.3ms	remaining: 14.6s
6:	learn: 5266107.3509078	total: 103ms	remaining: 14.6s
7:	learn: 5154473.2938151	total: 117ms	remaining: 14.5s
8:	learn: 5045786.1839582	total: 132ms	remaining: 14.5s
9:	learn: 4949712.0852960	total: 146ms	remaining: 14.4s
10:	learn: 4863107.1204516	total: 161ms	remaining: 14.5s
11:	learn: 4779720.3155783	total: 175ms	remaining: 14.4s
12:	learn: 4711397.5776012	total: 198ms	remaining: 15.1s
13:	learn: 4636218.0593268	total: 213ms	remaining: 15s
14:	learn: 4561193.0602113	total: 229ms	remaining: 15.1s
15:	learn: 4485478.7206993	total: 245ms	remaining: 15.1s
16:	learn: 4424565.7270561	total: 261ms	remaining: 15.1s
17:	learn: 43

In [122]:
print(mean_squared_error(y_pred=tree_reg.predict(X_test), y_true=y_test))
print(mean_squared_error(y_pred=forest_regressor.predict(X_test), y_true=y_test))
print(mean_squared_error(y_pred=cat.predict(X_test), y_true=y_test))
print(tree_reg.score(X_test, y_test))
print(forest_regressor.score(X_test, y_test))
print(cat.score(X_test, y_test))

18090917814411.332
13584903158636.066
12235422888660.348
0.4108878907929375
0.5576216179157856
0.6015660532581104


Лучше всего работает catboost.

In [149]:
grid = {'depth': [1, 2, 3, 4, 5, 7, 8, 9, 10],
        'iterations': [250, 100, 500, 1000],
        'learning_rate': [0.005, 0.001, 0.01, 0.1, 0.2, 0.3],
        'l2_leaf_reg': [1, 3, 5, 10, 100],
        'border_count': [5, 10, 20, 30, 50, 100, 200],
        }
model = catboost.CatBoostRegressor()
randomized_search_result = model.randomized_search(grid, X=X_train, y=y_train, plot=True)

best_model_params = randomized_search_result['params']
best_cat = catboost.CatBoostRegressor(random_state=42)
best_cat.set_params(**best_model_params)
best_cat.fit(X_train, y_train)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 8409770.6301751	test: 9039587.1110986	best: 9039587.1110986 (0)	total: 16.2ms	remaining: 1.6s
1:	learn: 6883524.3980507	test: 7502389.8473931	best: 7502389.8473931 (1)	total: 25.3ms	remaining: 1.24s
2:	learn: 5890894.3526234	test: 6462529.3375664	best: 6462529.3375664 (2)	total: 32.6ms	remaining: 1.05s
3:	learn: 5203345.2153678	test: 5732605.8259589	best: 5732605.8259589 (3)	total: 39.5ms	remaining: 949ms
4:	learn: 4743762.0017272	test: 5262845.3402702	best: 5262845.3402702 (4)	total: 49.7ms	remaining: 944ms
5:	learn: 4457300.7065805	test: 4996711.6842290	best: 4996711.6842290 (5)	total: 56.8ms	remaining: 890ms
6:	learn: 4161952.8149468	test: 4763222.8326763	best: 4763222.8326763 (6)	total: 63.9ms	remaining: 849ms
7:	learn: 3961847.7668743	test: 4535978.7683051	best: 4535978.7683051 (7)	total: 70.7ms	remaining: 813ms
8:	learn: 3815471.7310322	test: 4400134.1826923	best: 4400134.1826923 (8)	total: 94.9ms	remaining: 959ms
9:	learn: 3703537.4712082	test: 4310736.5717064	best: 43

In [150]:
print(best_cat.score(X_test, y_test))


0.5396785108312736


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [129]:
df = pd.get_dummies(df, columns=cat_columns)


In [130]:
data_invest = df[df["product_type_Investment"] == 1]
data_invest.columns.get_loc('product_type_Investment')


274

In [138]:
X_invest = data_invest.drop(columns=['price', 'timestamp'])
X_invest = np.array(X_invest)
y_invest = np.array(data_invest['price'])


In [ ]:
data_owner = df[df["product_type_Investment"] == 0]
X_owner = data_owner.drop(columns=['price', 'timestamp'])
X_owner = np.array(X_owner)
y_owner = np.array(data_owner['price'])

In [ ]:
invest = catboost.CatBoostRegressor(random_state=42)
invest.set_params(**best_model_params)
invest.fit(X_invest[:, 1:], y_invest)
owner = catboost.CatBoostRegressor(random_state=0)
owner.set_params(**best_model_params)
owner.fit(X_owner[:, 1:], y_owner)

In [145]:
lst = []
for i in range(X_test.shape[0]):
    if X_test[i, 274] == 1:
        lst.append(invest.predict(X_test[i, :].reshape(1, X_test.shape[1])))
    else:
        lst.append(owner.predict(X_test[i, :].reshape(1, X_test.shape[1])))


In [151]:
print(mean_squared_error(y_test, lst))

# прошлый результат
print(mean_squared_error(y_test, best_cat.predict(X_test)))


13477183146949.816
14135914198008.525


Стало немного лучше

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).